<a href="https://colab.research.google.com/github/messias077/REP/blob/main/Problema_Generalizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
""" Avaliando a generalização de algoritmos """

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_squared_error, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
from sklearn.datasets import load_iris

# Carregando os dados
wine = load_wine()
X, y = wine.data, wine.target

#iris = load_iris()
#X = iris.data
#y = iris.target

kf = KFold(n_splits=10)

# Logistic Regression convencional
lr_conv = LogisticRegression(max_iter=10000)

# Logistic Regression aninhado
scaler = StandardScaler()

lr_anin = Pipeline([
    ("Padronização", scaler),
    ("classificador", LogisticRegression(max_iter=3000))
])

# KNN convencional
knn_conv = KNeighborsRegressor()

# KNN Aninhado
parametros = {'n_neighbors': [3, 5, 7]}

knn_anin = GridSearchCV(KNeighborsRegressor(), parametros, 
                      scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))


# Guarda as acurácias de cada uma das rodadas para calcular a média
acuracias_lr_conv = []
acuracias_lr_anin = []
acuracias_knn_conv = []
acuracias_knn_anin = []

# Rodando todas os n_splits do KFold
for treinamento, teste in kf.split(X, y):
    X_treinamento = X[treinamento]
    y_treinamento = y[treinamento]

    X_teste = X[teste]
    y_teste = y[teste]

    lr_conv.fit(X_treinamento, y_treinamento)
    y_pred_lr_conv = lr_conv.predict(X_teste)
    acur_lr_conv = accuracy_score(y_teste, y_pred_lr_conv)
    acuracias_lr_conv.append(acur_lr_conv)

    lr_anin.fit(X_treinamento, y_treinamento)
    y_pred_lr_anin = lr_conv.predict(X_teste)
    acur_lr_anin = accuracy_score(y_teste, y_pred_lr_conv)
    acuracias_lr_anin.append(acur_lr_anin)

    knn_conv.fit(X_treinamento, y_treinamento)
    y_pred_knn_conv = knn_conv.predict(X_teste).astype(int)
    acur_knn_conv = accuracy_score(y_teste, y_pred_knn_conv)
    acuracias_knn_conv.append(acur_knn_conv)

    knn_anin.fit(X_treinamento, y_treinamento)
    y_pred_knn_anin = knn_anin.predict(X_teste).astype(int)
    acur_knn_anin = accuracy_score(y_teste, y_pred_knn_anin)
    acuracias_knn_anin.append(acur_knn_anin)

print("\n  --------------------------------------")
print("\n         **** Resultados ****")
print("\n  --------------------------------------")

print("\n* K-Fold Convencional")
print("\n - Logistic Regression\n")
print(f"   Acurácia global..: {np.mean(acuracias_lr_conv)}")
print(f"   Desvio padrão....: {np.std(acuracias_lr_conv)}")

print("\n - KNeighbors Regressor\n")
print(f"   Acurácia global..: {np.mean(acuracias_knn_conv)}")
print(f"   Desvio padrão....: {np.std(acuracias_knn_conv)}")

print("\n\n* K-Fold Aninhado")
print("\n - Logistic Regression\n")
print(f"   Acurácia global..: {np.mean(acuracias_lr_anin)}")
print(f"   Desvio padrão....: {np.std(acuracias_lr_anin)}")

print("\n - KNeighbors Regressor\n")
print(f"   Acurácia global..: {np.mean(acuracias_knn_anin)}")
print(f"   Desvio padrão....: {np.std(acuracias_knn_anin)}")



  --------------------------------------

         **** Resultados ****

  --------------------------------------

* K-Fold Convencional

 - Logistic Regression

   Acurácia global..: 0.95
   Desvio padrão....: 0.05241100628920338

 - KNeighbors Regressor

   Acurácia global..: 0.6333333333333333
   Desvio padrão....: 0.37531879453454525


* K-Fold Aninhado

 - Logistic Regression

   Acurácia global..: 0.95
   Desvio padrão....: 0.05241100628920338

 - KNeighbors Regressor

   Acurácia global..: 0.65
   Desvio padrão....: 0.38813418832685703
